# Code Description: Data Cleaning and Preprocessing Script

## Purpose
This script is designed to perform initial data cleaning and preprocessing steps on the raw violence data (either 'Casos' or 'Victimas') for the project on Selective and Indiscriminate Violence (VS/VI) in Colombia. Its main goal is to prepare the data for subsequent analysis, metric calculation (Escalation, Intensity), and potential modeling.

## Workflow Stage
This script is in the Data Cleaning / Preprocessing stage. It takes the raw data, likely loaded from the combined DataFrame generated in the previous step, and transforms it into a clean, structured format suitable for further use in the analytical pipeline.

## About
This script will handle common data issues such as missing values, incorrect data types, and inconsistencies. It will standardize column names if necessary and potentially aggregate data by relevant temporal (e.g., month) and geographical (Country, Department, or Region) units, depending on the specific analysis level being targeted. The output will be a cleaned dataset ready for calculating metrics and generating features.


In [1]:
import pandas as pd
import numpy as np
import os
from itertools import product 

/var/folders/6q/v6tkj0h970nd7vm3v0d4zvm80000gn/T/ipykernel_5255/2357270746.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### 1. Initial Setup, Library Imports, and Path Configuration
This block performs the initial setup, including importing necessary libraries (pandas, os), defining the path to the raw data folder (one level up in 'Data/raw'), and listing the specific filenames expected for VI and VS violence types. It also defines the list of columns to be extracted from each file.

In [2]:
# Define the path to the folder containing the Excel files.
# Assumes the notebook is in a subfolder and the data is one level up.
# Adjust '../' if your folder structure is different.
data_folder_path = os.path.join(os.getcwd(), '..', 'Data', 'raw','cases') # Example path based on the proposed structure

# Define the list of columns to select from each Excel file.
columns_to_select = [
    "Año",
    "Mes",
    "Día",
    "ID Caso",
    "Municipio",
    "Departamento",
    "Región"
]

# Define the lists of filenames corresponding to each violence type (VI and VS).
# These filenames are used to classify the data.
vi_files = [
    "Casos_Acciones_Belicas_202503.xlsx",
    "Casos_Ataques_a_Poblaciones_202503.xlsx",
    "Casos_Atentados_Terroristas_202503.xlsx",
    "Casos_MInas_202503.xlsx",
    "Casos_Reclutamiento_ninas_ninos_U_202503.xlsx"
]

vs_files = [
    "Caso_ Danos_a_Bienes_Civiles_202503.xlsx", # Note: Check for potential extra space in filename "Caso_ Danos..."
    "Casos_Asesinatos_Selectivo_202503.xlsx",
    "Casos_Desaparicion_Forzada _202503.xlsx", # Note: Check for potential extra space in filename "Desaparicion_Forzada _"
    "Casos_Masacre_202503.xlsx",
    "Casos_Secuestro_202503.xlsx",
    "Casos_Violencia_Sexual_202503.xlsx"
]

# Initialize an empty list to store the processed dataframes from each file.
all_dataframes = []

# Iterate through all files in the specified data folder.
for filename in os.listdir(data_folder_path):
    # Construct the full file path.
    file_path = os.path.join(data_folder_path, filename)

    # Check if the current item is a file and if it's an Excel file.
    if os.path.isfile(file_path) and filename.endswith('.xlsx'):
        print(f"Processing file: {filename}") # Print the filename being processed

        try:
            # Read the Excel file into a pandas DataFrame.
            df = pd.read_excel(file_path)

            # Select only the required columns.
            # Use .copy() to avoid SettingWithCopyWarning later.
            df_selected = df[columns_to_select].copy()

            # Determine the violence type based on the filename and add the 'violence type' column.
            if filename in vi_files:
                df_selected['violence type'] = 'VI'
            elif filename in vs_files:
                df_selected['violence type'] = 'VS'
            else:
                # If the file is not in either list, you might want to skip it
                # or assign a different type, depending on your needs.
                print(f"Warning: File '{filename}' not classified as VI or VS. Skipping.")
                continue # Skip this file

            # Append the processed DataFrame to the list.
            all_dataframes.append(df_selected)

        except Exception as e:
            # Print an error message if reading or processing a file fails.
            print(f"Error processing file {filename}: {e}")

Processing file: Casos_Desaparicion_Forzada _202503.xlsx


/Users/diegohernandez/Documents/Virtual_Enviroment/Python_Colab/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Processing file: Casos_Reclutamiento_ninas_ninos_U_202503.xlsx


/Users/diegohernandez/Documents/Virtual_Enviroment/Python_Colab/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Processing file: Casos_Acciones_Belicas_202503.xlsx


...

/Users/diegohernandez/Documents/Virtual_Enviroment/Python_Colab/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/diegohernandez/Documents/Virtual_Enviroment/Python_Colab/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Processing file: Casos_Secuestro_202503.xlsx


/Users/diegohernandez/Documents/Virtual_Enviroment/Python_Colab/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Processing file: Casos_Masacre_202503.xlsx


/Users/diegohernandez/Documents/Virtual_Enviroment/Python_Colab/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


### 2. Concatenate DataFrames and Display Summary
This block consolidates all individual DataFrames processed from the Excel files into a single combined_df. It includes a check to ensure data was processed before concatenation. Finally, it displays the head, info, and violence type counts of the combined DataFrame for initial verification.

In [3]:
# Concatenate all dataframes in the list into a single DataFrame.
# ignore_index=True resets the index of the resulting dataframe.
if all_dataframes:
    combined_df = pd.concat(all_dataframes, ignore_index=True)

    # Display the first few rows of the combined DataFrame.
    print("\nCombined DataFrame Head:")
    print(combined_df.head())

    # Display information about the combined DataFrame (column types, non-null counts).
    print("\nCombined DataFrame Info:")
    combined_df.info()

    # Optional: Display value counts for the 'violence type' column to verify classification.
    print("\nViolence Type Counts:")
    print(combined_df['violence type'].value_counts())

else:
    print("\nNo Excel files were processed or found.")


Combined DataFrame Head:
    Año  Mes  Día  ID Caso  Municipio  Departamento  \
0  1991    5   25   100265  JERUSALEN  CUNDINAMARCA   
1  2004   12    2   100282    LA MESA  CUNDINAMARCA   
2  1993    3    9   101616     YACOPI  CUNDINAMARCA   
3  1997    6    8   102204     BOJAYA         CHOCO   
4  2000    6   12   102489      LLORO         CHOCO   

                         Región violence type  
0  SUROCCIDENTE DE CUNDINAMARCA            VS  
1  SUROCCIDENTE DE CUNDINAMARCA            VS  
2               MAGDALENA MEDIO            VS  
3                        ATRATO            VS  
4                        ATRATO            VS  

Combined DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368370 entries, 0 to 368369
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Año            368370 non-null  int64 
 1   Mes            368370 non-null  int64 
 2   Día            368370 non-null  int64 

In [4]:
combined_df.to_csv('../Data/processed/cases/real_total_cases.tsv',sep='\t')
combined_df.head()

,Año,Mes,Día,ID Caso,Municipio,Departamento,Región,violence type
0,1991,5,25,100265,JERUSALEN,CUNDINAMARCA,SUROCCIDENTE DE CUNDINAMARCA,VS
1,2004,12,2,100282,LA MESA,CUNDINAMARCA,SUROCCIDENTE DE CUNDINAMARCA,VS
2,1993,3,9,101616,YACOPI,CUNDINAMARCA,MAGDALENA MEDIO,VS
3,1997,6,8,102204,BOJAYA,CHOCO,ATRATO,VS
4,2000,6,12,102489,LLORO,CHOCO,ATRATO,VS


### 3. Group Cases Data by Year, Month, and Violence Type (1958-2022)
This block filters the combined_df for the years 1958-2022 and groups it by year, month, and 'violence type' to count the total number of cases for each type (VI and VS). It ensures all possible Year-Month-Violence Type combinations within this range are present by filling missing entries with a count of 0, creating a complete monthly time series for cases per violence type at the country level.

In [5]:
# Ensure combined_df exists from the previous step
if 'combined_df' not in locals():
    print("Error: 'combined_df' not found. Please run the previous code block to create it.")
    combined_df = pd.read_csv('../Data/processed/cases/real_total_cases.tsv')
else:
    # --- 3. Group Cases Data by Year, Month, and Violence Type (1958-2022) for Country Level ---

    print("\n--- Grouping Cases data by Year, Month, and Violence Type for Country (1958-2022) ---")

    # 1. Select only records from the years 1958 to 2022.
    # Ensure 'Año' column is numeric, handling potential errors.
    try:
        # Make a copy to avoid modifying the original combined_df directly
        df_filtered_years_cases = combined_df.copy()
        df_filtered_years_cases['Año'] = pd.to_numeric(df_filtered_years_cases['Año'], errors='coerce')
        # Drop rows where 'Año' could not be converted to a number (NaN)
        df_filtered_years_cases = df_filtered_years_cases.dropna(subset=['Año']).copy()

        # Filter by the specified year range
        df_filtered_years_cases = df_filtered_years_cases[
            (df_filtered_years_cases['Año'] >= 1958) & (df_filtered_years_cases['Año'] <= 2022)
        ].copy()

        print(f"Filtered cases data for years 1958-2022. Shape: {df_filtered_years_cases.shape}")

        # Ensure 'Mes' column is numeric, handling potential errors.
        df_filtered_years_cases['Mes'] = pd.to_numeric(df_filtered_years_cases['Mes'], errors='coerce')
        # Drop rows where 'Mes' could not be converted to a number (NaN)
        df_filtered_years_cases = df_filtered_years_cases.dropna(subset=['Mes']).copy()

        # Ensure 'violence type' column exists and is not empty
        if 'violence type' not in df_filtered_years_cases.columns or df_filtered_years_cases['violence type'].isnull().all():
             print("Error: 'violence type' column is missing or empty. Please check the previous data loading step.")
        else:
            # 2. Generate a group by for each month within each year and violence type to count the number of cases (rows).
            # Group by 'Año', 'Mes', and 'violence type' and count the occurrences (size of each group).
            # The result is a pandas Series with multi-index (Año, Mes, violence type).
            cases_by_month_year_type_country = df_filtered_years_cases.groupby(['Año', 'Mes', 'violence type']).size()

            print("\nCases by Month, Year, and Violence Type (Country Level - partial view):")
            print(cases_by_month_year_type_country.head())
            print("...")
            print(cases_by_month_year_type_country.tail())


            # 3. Ensure each year from 1958 to 2022 has all 12 months for each violence type.
            # Create a complete list of all expected Year-Month-Violence Type combinations.
            # Get all unique violence types present in the data
            unique_violence_types = df_filtered_years_cases['violence type'].unique()

            # Create a complete list of all expected Year-Month combinations.
            full_date_range_cases = pd.date_range(start='1958-01-01', end='2022-12-01', freq='MS') # Month Start frequency

            # Create a list of all possible (Year, Month, Violence Type) combinations
            all_combinations = list(product(full_date_range_cases.year, full_date_range_cases.month, unique_violence_types))

            # Create a MultiIndex from all combinations
            full_year_month_type_index_cases = pd.MultiIndex.from_tuples(
                all_combinations,
                names=['Año', 'Mes', 'violence type']
            )

            # Reindex the cases_by_month_year_type_country Series using the complete index.
            # This will add missing Year-Month-Violence Type combinations with NaN values.
            cases_by_month_year_complete_country = cases_by_month_year_type_country.reindex(full_year_month_type_index_cases)

            # Fill the NaN values (for months/types with no cases) with 0.
            cases_by_month_year_complete_country = cases_by_month_year_complete_country.fillna(0).astype(int)

            # Convert the Series back to a DataFrame for easier handling.
            # The column name will be 'CaseCount'.
            grouped_cases_country_monthly_type = cases_by_month_year_complete_country.reset_index(name='CaseCount')

            # Sort the DataFrame by Year, Month, and Violence Type to ensure chronological order.
            grouped_cases_country_monthly_type = grouped_cases_country_monthly_type.sort_values(by=['Año', 'Mes', 'violence type']).reset_index(drop=True)


            print("\nGrouped Cases DataFrame (Country Level - with imputed months and types):")
            print(grouped_cases_country_monthly_type.head())
            print("...")
            print(grouped_cases_country_monthly_type.tail())

            print("\nInfo of Grouped Cases DataFrame (Country Level):")
            grouped_cases_country_monthly_type.info()

            print("\nCases data grouping and imputation for Country level complete.")
            print("Resulting DataFrame stored in 'grouped_cases_country_monthly_type'.")

    except KeyError as e:
        print(f"Error: Required column not found - {e}. Please check column names in the combined_df.")
    except Exception as e:
        print(f"An unexpected error occurred during grouping: {e}")




--- Grouping Cases data by Year, Month, and Violence Type for Country (1958-2022) ---
Filtered cases data for years 1958-2022. Shape: (345266, 8)

Cases by Month, Year, and Violence Type (Country Level - partial view):
Año   Mes  violence type
1958  0    VI                3
           VS               25
      1    VI               10
           VS               91
      2    VI               13
dtype: int64
...
Año   Mes  violence type
2022  10   VS               68
      11   VI               22
           VS               62
      12   VI               30
           VS               41
dtype: int64

Grouped Cases DataFrame (Country Level - with imputed months and types):
    Año  Mes violence type  CaseCount
0  1958    1            VI         10
1  1958    1            VI         10
2  1958    1            VI         10
3  1958    1            VI         10
4  1958    1            VI         10
...
          Año  Mes violence type  CaseCount
1216795  2022   12            VS        

In [6]:
grouped_cases_country_monthly_type.to_csv('../Data/processed/cases/country/1958_2022_cases_country.tsv',sep='\t')
grouped_cases_country_monthly_type.head()

,Año,Mes,violence type,CaseCount
0,1958,1,VI,10
1,1958,1,VI,10
2,1958,1,VI,10
3,1958,1,VI,10
4,1958,1,VI,10


### 4. Generate Animated Line Plot of VI vs VS Cases (1958-2022)
This block creates an animated line plot visualizing the yearly trend of Selective Violence (VS) and Indiscriminate Violence (VI) cases in Colombia from 1958 to 2022. The animation shows how the cumulative case counts for each violence type evolve over time, providing a dynamic view of their historical trajectories. The output is saved as an MP4 video file.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import seaborn as sns # Import seaborn for aesthetics
from itertools import product # Import product for generating combinations (if needed for re-running previous cells)

grouped_cases_country_monthly_type = pd.read_csv('../Data/processed/cases/country/1958_2022_cases_country.tsv',sep='\t')


# Ensure the grouped_cases_country_monthly_type DataFrame exists
if 'grouped_cases_country_monthly_type' not in locals():
    print("Error: 'grouped_cases_country_monthly_type' not found. Please run the previous code block to create it.")
else:
    # --- 4. Generate Animated Line Plot of VI vs VS Cases (1958-2022) ---

    print("\n--- Generating Animated Line Plot ---")

    # Prepare data: Group by Year and Violence Type and sum the monthly counts
    # This gives the total cases per year for each violence type
    yearly_cases_by_type = grouped_cases_country_monthly_type.groupby(['Año', 'violence type'])['CaseCount'].sum().reset_index()

    # Pivot the data for easier plotting
    # Years will be the index, violence types will be columns, and values will be CaseCount
    yearly_cases_pivot = yearly_cases_by_type.pivot(index='Año', columns='violence type', values='CaseCount').fillna(0)

    # Ensure both 'VI' and 'VS' columns exist, even if one had 0 cases for all years
    for v_type in ['VI', 'VS']:
        if v_type not in yearly_cases_pivot.columns:
            yearly_cases_pivot[v_type] = 0

    # Sort the pivot table by year
    yearly_cases_pivot = yearly_cases_pivot.sort_index()

    # Calculate cumulative sum for the animation
    # This shows the total cases up to a given year
    yearly_cases_cumulative = yearly_cases_pivot.cumsum()

    # Set up the figure and axes for the plot
    plt.style.use('seaborn-v0_8-darkgrid') # Use a nice seaborn style
    fig, ax = plt.subplots(figsize=(12, 7))

    # Set initial plot limits (adjust as needed)
    ax.set_xlim(yearly_cases_cumulative.index.min(), yearly_cases_cumulative.index.max())
    ax.set_ylim(0, yearly_cases_cumulative.values.max() * 1.1) # Add 10% padding to y-axis

    # Set titles and labels
    ax.set_title('Cumulative Cases of Selective (VS) and Indiscriminate (VI) Violence in Colombia (1958-2022)', fontsize=14)
    ax.set_xlabel('Year', fontsize=12)
    ax.set_ylabel('Cumulative Number of Cases', fontsize=12)
    ax.grid(True, linestyle='--', alpha=0.6)

    # Initialize the lines for the plot
    line_vi, = ax.plot([], [], label='VI Cases', color='red', linewidth=2)
    line_vs, = ax.plot([], [], label='VS Cases', color='blue', linewidth=2)
    # --- ADJUSTMENT HERE: Change loc to 'lower right' ---
    ax.legend(loc='lower right')

    # Add a text annotation for the current year (will be updated in animation)
    year_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, fontsize=15, color='gray')

    # Define the animation update function
    def update(frame):
        """
        Updates the plot data for each frame of the animation.
        frame: The current frame number (index of the year).
        """
        current_year_index = frame
        current_year = yearly_cases_cumulative.index[current_year_index]

        # Update data for VI line up to the current year
        line_vi.set_data(yearly_cases_cumulative.index[:current_year_index+1],
                         yearly_cases_cumulative['VI'].iloc[:current_year_index+1])

        # Update data for VS line up to the current year
        line_vs.set_data(yearly_cases_cumulative.index[:current_year_index+1],
                         yearly_cases_cumulative['VS'].iloc[:current_year_index+1])

        # Update the year text annotation
        year_text.set_text(f'Year: {current_year}')

        return line_vi, line_vs, year_text, ax.legend_

    # Create the animation
    # frames: number of frames (equal to the number of years)
    # interval: delay between frames in milliseconds
    # blit: True means only re-draw the parts that have changed (can be faster)
    ani = animation.FuncAnimation(fig, update, frames=len(yearly_cases_cumulative.index),
                                  interval=200, blit=True) # Adjust interval for speed

    # Define the output path for the video
    output_dir = os.path.join(os.getcwd(), '..', 'Images')
    output_filename = 'VI_VS_Colombia.mp4'
    output_path = os.path.join(output_dir, output_filename)

    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Save the animation
    # Requires ffmpeg. If you don't have it, you might need to install it
    # (e.g., using conda install -c conda-forge ffmpeg or through a system package manager)
    try:
        print(f"\nSaving animation to {output_path}...")
        # Fix for Matplotlib Deprecation Warning: close figure before switching backend
        plt.close(fig) # Close the figure before switching
        plt.switch_backend('agg') # Switch backend for saving

        # Re-create the writer after closing and switching
        writer = animation.FFMpegWriter(fps=10) # frames per second
        # Re-create the animation object, or ensure the writer can handle the original fig
        # It's often better to just pass the fig directly to save if possible,
        # or ensure the backend is set correctly BEFORE figure creation if saving without displaying.
        # However, since the figure is needed for FuncAnimation, closing *before* saving
        # is the fix for the specific warning. Let's try saving the *original* animation object.
        ani.save(output_path, writer=writer)

        print("Animation saved successfully!")
    except Exception as e:
        print(f"\nError saving animation: {e}")
        print("Please ensure you have ffmpeg installed and accessible in your environment.")
        print("You might need to install it using: conda install -c conda-forge ffmpeg")
        print("Or using your system's package manager (e.g., sudo apt-get install ffmpeg on Ubuntu or brew install ffmpeg on macOS).")





--- Generating Animated Line Plot ---

Saving animation to /Users/diegohernandez/Documents/GitHub/VS_VI_Source_Code/Scripts/../Images/VI_VS_Colombia.mp4...
Animation saved successfully!
